In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets

In [2]:
# train에 필요한 parameter 설정하기

lr = 1e-3
batch_size = 64
num_epoch = 10

# 학습된 data가 저장될 dir
ckpt_dir = './checkpoint'
# tensorboard가 저장될 dir
log_dir = './log'
# device 정보 입력
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# classifier network 구축

class Net(nn.Module) :
    # Layer 초기화 단계
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=0, bias=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1, padding=0, bias=True)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=320, out_features=50, bias=True)
        self.relu1_fc1 = nn.ReLU()
        self.drop1_fc1 = nn.Dropout2d(p=0.5)

        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)
        
        
    # Layer 구축 단계
    def forward(self,x) :
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.relu2(x)
    
        
        x = x.view(-1, 320)

        x = self.fc1(x)
        x = self.relu1_fc1(x)
        x = self.drop1_fc1(x)

        x = self.fc2(x)
        
        return x

In [4]:
# 네트워크를 저장하거나 불러오는 함수 (save(), load())

def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
        
    torch.save({'net':net.state_dict(), 'optim': optim.state_dict()}, 
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))
    
def load(ckpt_dir, net, optim) :
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort()
    
    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))
    
    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dicc_model['optim'])
    
    return net, optim

In [5]:
# MNIST 데이터 셋 불러오기

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

dataset = datasets.MNIST(download=True, root='./', train=True, transform = transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

num_data = len(loader.dataset)
num_batch = np.ceil(num_data / batch_size)

In [6]:
# 네트워크 설정 및 필요한 손실함수 구현하기
net = Net().to(device)
params = net.parameters()

fn_loss = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output: torch.softmax(output, dim=1)
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()

optim = torch.optim.Adam(params, lr=lr)

writer = SummaryWriter(log_dir=log_dir)

In [7]:
# 네트워크 학습을 위한 for문 구현 (트레이닝 시작)

for epoch in range(1, num_epoch + 1) :
    net.train()
    
    loss_arr = []
    acc_arr = []
    
    for batch, (input, label) in enumerate(loader, 1) :
        input = input.to(device)
        label = label.to(device)
        
        output = net(input)
        pred = fn_pred(output)
        
        optim.zero_grad()
        
        loss = fn_loss(output, label)
        acc = fn_acc(pred, label)
        
        loss.backward()
        
        optim.step()
        
        loss_arr += [loss.item()]
        acc_arr += [acc.item()]
        
        print('TRAIN: EPOCH %04d/%04d | BATCH %04d/%04d | LOSS %.4f | ACC %.4f' % 
              (epoch, num_epoch, batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

    writer.add_scalar('loss', np.mean(loss_arr),epoch)
    writer.add_scalar('loss', np.mean(acc_arr),epoch)
    
    save(ckpt_dir = ckpt_dir, net =net, optim = optim, epoch = epoch)

writer.close()

/home/blue/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


TRAIN: EPOCH 0001/0010 | BATCH 0001/0938 | LOSS 2.3257 | ACC 0.1094
TRAIN: EPOCH 0001/0010 | BATCH 0002/0938 | LOSS 2.3213 | ACC 0.1094
TRAIN: EPOCH 0001/0010 | BATCH 0003/0938 | LOSS 2.3250 | ACC 0.0990
TRAIN: EPOCH 0001/0010 | BATCH 0004/0938 | LOSS 2.3201 | ACC 0.0977
TRAIN: EPOCH 0001/0010 | BATCH 0005/0938 | LOSS 2.3170 | ACC 0.1187
TRAIN: EPOCH 0001/0010 | BATCH 0006/0938 | LOSS 2.3155 | ACC 0.1172
TRAIN: EPOCH 0001/0010 | BATCH 0007/0938 | LOSS 2.3155 | ACC 0.1094
TRAIN: EPOCH 0001/0010 | BATCH 0008/0938 | LOSS 2.3094 | ACC 0.1152
TRAIN: EPOCH 0001/0010 | BATCH 0009/0938 | LOSS 2.3061 | ACC 0.1198
TRAIN: EPOCH 0001/0010 | BATCH 0010/0938 | LOSS 2.3058 | ACC 0.1203
TRAIN: EPOCH 0001/0010 | BATCH 0011/0938 | LOSS 2.3043 | ACC 0.1193
TRAIN: EPOCH 0001/0010 | BATCH 0012/0938 | LOSS 2.3025 | ACC 0.1211
TRAIN: EPOCH 0001/0010 | BATCH 0013/0938 | LOSS 2.3016 | ACC 0.1238
TRAIN: EPOCH 0001/0010 | BATCH 0014/0938 | LOSS 2.3012 | ACC 0.1239
TRAIN: EPOCH 0001/0010 | BATCH 0015/0938 | LOSS 

TRAIN: EPOCH 0001/0010 | BATCH 0132/0938 | LOSS 1.5145 | ACC 0.4766
TRAIN: EPOCH 0001/0010 | BATCH 0133/0938 | LOSS 1.5083 | ACC 0.4783
TRAIN: EPOCH 0001/0010 | BATCH 0134/0938 | LOSS 1.5027 | ACC 0.4802
TRAIN: EPOCH 0001/0010 | BATCH 0135/0938 | LOSS 1.4997 | ACC 0.4814
TRAIN: EPOCH 0001/0010 | BATCH 0136/0938 | LOSS 1.4941 | ACC 0.4836
TRAIN: EPOCH 0001/0010 | BATCH 0137/0938 | LOSS 1.4893 | ACC 0.4851
TRAIN: EPOCH 0001/0010 | BATCH 0138/0938 | LOSS 1.4851 | ACC 0.4863
TRAIN: EPOCH 0001/0010 | BATCH 0139/0938 | LOSS 1.4795 | ACC 0.4881
TRAIN: EPOCH 0001/0010 | BATCH 0140/0938 | LOSS 1.4737 | ACC 0.4903
TRAIN: EPOCH 0001/0010 | BATCH 0141/0938 | LOSS 1.4691 | ACC 0.4925
TRAIN: EPOCH 0001/0010 | BATCH 0142/0938 | LOSS 1.4651 | ACC 0.4939
TRAIN: EPOCH 0001/0010 | BATCH 0143/0938 | LOSS 1.4602 | ACC 0.4953
TRAIN: EPOCH 0001/0010 | BATCH 0144/0938 | LOSS 1.4567 | ACC 0.4967
TRAIN: EPOCH 0001/0010 | BATCH 0145/0938 | LOSS 1.4510 | ACC 0.4994
TRAIN: EPOCH 0001/0010 | BATCH 0146/0938 | LOSS 

TRAIN: EPOCH 0001/0010 | BATCH 0263/0938 | LOSS 1.0634 | ACC 0.6422
TRAIN: EPOCH 0001/0010 | BATCH 0264/0938 | LOSS 1.0610 | ACC 0.6431
TRAIN: EPOCH 0001/0010 | BATCH 0265/0938 | LOSS 1.0590 | ACC 0.6438
TRAIN: EPOCH 0001/0010 | BATCH 0266/0938 | LOSS 1.0566 | ACC 0.6446
TRAIN: EPOCH 0001/0010 | BATCH 0267/0938 | LOSS 1.0554 | ACC 0.6450
TRAIN: EPOCH 0001/0010 | BATCH 0268/0938 | LOSS 1.0541 | ACC 0.6453
TRAIN: EPOCH 0001/0010 | BATCH 0269/0938 | LOSS 1.0521 | ACC 0.6457
TRAIN: EPOCH 0001/0010 | BATCH 0270/0938 | LOSS 1.0504 | ACC 0.6466
TRAIN: EPOCH 0001/0010 | BATCH 0271/0938 | LOSS 1.0483 | ACC 0.6476
TRAIN: EPOCH 0001/0010 | BATCH 0272/0938 | LOSS 1.0460 | ACC 0.6484
TRAIN: EPOCH 0001/0010 | BATCH 0273/0938 | LOSS 1.0435 | ACC 0.6492
TRAIN: EPOCH 0001/0010 | BATCH 0274/0938 | LOSS 1.0413 | ACC 0.6500
TRAIN: EPOCH 0001/0010 | BATCH 0275/0938 | LOSS 1.0388 | ACC 0.6509
TRAIN: EPOCH 0001/0010 | BATCH 0276/0938 | LOSS 1.0375 | ACC 0.6514
TRAIN: EPOCH 0001/0010 | BATCH 0277/0938 | LOSS 

TRAIN: EPOCH 0001/0010 | BATCH 0395/0938 | LOSS 0.8599 | ACC 0.7140
TRAIN: EPOCH 0001/0010 | BATCH 0396/0938 | LOSS 0.8585 | ACC 0.7144
TRAIN: EPOCH 0001/0010 | BATCH 0397/0938 | LOSS 0.8572 | ACC 0.7149
TRAIN: EPOCH 0001/0010 | BATCH 0398/0938 | LOSS 0.8565 | ACC 0.7151
TRAIN: EPOCH 0001/0010 | BATCH 0399/0938 | LOSS 0.8558 | ACC 0.7153
TRAIN: EPOCH 0001/0010 | BATCH 0400/0938 | LOSS 0.8548 | ACC 0.7156
TRAIN: EPOCH 0001/0010 | BATCH 0401/0938 | LOSS 0.8533 | ACC 0.7162
TRAIN: EPOCH 0001/0010 | BATCH 0402/0938 | LOSS 0.8530 | ACC 0.7164
TRAIN: EPOCH 0001/0010 | BATCH 0403/0938 | LOSS 0.8517 | ACC 0.7168
TRAIN: EPOCH 0001/0010 | BATCH 0404/0938 | LOSS 0.8511 | ACC 0.7170
TRAIN: EPOCH 0001/0010 | BATCH 0405/0938 | LOSS 0.8496 | ACC 0.7176
TRAIN: EPOCH 0001/0010 | BATCH 0406/0938 | LOSS 0.8488 | ACC 0.7180
TRAIN: EPOCH 0001/0010 | BATCH 0407/0938 | LOSS 0.8480 | ACC 0.7183
TRAIN: EPOCH 0001/0010 | BATCH 0408/0938 | LOSS 0.8466 | ACC 0.7187
TRAIN: EPOCH 0001/0010 | BATCH 0409/0938 | LOSS 

...

TRAIN: EPOCH 0010/0010 | BATCH 0401/0938 | LOSS 0.1363 | ACC 0.9597
TRAIN: EPOCH 0010/0010 | BATCH 0402/0938 | LOSS 0.1361 | ACC 0.9598
TRAIN: EPOCH 0010/0010 | BATCH 0403/0938 | LOSS 0.1364 | ACC 0.9596
TRAIN: EPOCH 0010/0010 | BATCH 0404/0938 | LOSS 0.1363 | ACC 0.9596
TRAIN: EPOCH 0010/0010 | BATCH 0405/0938 | LOSS 0.1361 | ACC 0.9597
TRAIN: EPOCH 0010/0010 | BATCH 0406/0938 | LOSS 0.1365 | ACC 0.9596
TRAIN: EPOCH 0010/0010 | BATCH 0407/0938 | LOSS 0.1364 | ACC 0.9596
TRAIN: EPOCH 0010/0010 | BATCH 0408/0938 | LOSS 0.1364 | ACC 0.9596
TRAIN: EPOCH 0010/0010 | BATCH 0409/0938 | LOSS 0.1367 | ACC 0.9595
TRAIN: EPOCH 0010/0010 | BATCH 0410/0938 | LOSS 0.1367 | ACC 0.9595
TRAIN: EPOCH 0010/0010 | BATCH 0411/0938 | LOSS 0.1367 | ACC 0.9594
TRAIN: EPOCH 0010/0010 | BATCH 0412/0938 | LOSS 0.1365 | ACC 0.9595
TRAIN: EPOCH 0010/0010 | BATCH 0413/0938 | LOSS 0.1364 | ACC 0.9594
TRAIN: EPOCH 0010/0010 | BATCH 0414/0938 | LOSS 0.1365 | ACC 0.9594
TRAIN: EPOCH 0010/0010 | BATCH 0415/0938 | LOSS 

TRAIN: EPOCH 0010/0010 | BATCH 0527/0938 | LOSS 0.1381 | ACC 0.9588
TRAIN: EPOCH 0010/0010 | BATCH 0528/0938 | LOSS 0.1381 | ACC 0.9588
TRAIN: EPOCH 0010/0010 | BATCH 0529/0938 | LOSS 0.1382 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0530/0938 | LOSS 0.1382 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0531/0938 | LOSS 0.1381 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0532/0938 | LOSS 0.1381 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0533/0938 | LOSS 0.1381 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0534/0938 | LOSS 0.1381 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0535/0938 | LOSS 0.1380 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0536/0938 | LOSS 0.1382 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0537/0938 | LOSS 0.1381 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0538/0938 | LOSS 0.1381 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0539/0938 | LOSS 0.1380 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0540/0938 | LOSS 0.1378 | ACC 0.9588
TRAIN: EPOCH 0010/0010 | BATCH 0541/0938 | LOSS 

TRAIN: EPOCH 0010/0010 | BATCH 0652/0938 | LOSS 0.1374 | ACC 0.9589
TRAIN: EPOCH 0010/0010 | BATCH 0653/0938 | LOSS 0.1379 | ACC 0.9589
TRAIN: EPOCH 0010/0010 | BATCH 0654/0938 | LOSS 0.1379 | ACC 0.9589
TRAIN: EPOCH 0010/0010 | BATCH 0655/0938 | LOSS 0.1378 | ACC 0.9588
TRAIN: EPOCH 0010/0010 | BATCH 0656/0938 | LOSS 0.1378 | ACC 0.9588
TRAIN: EPOCH 0010/0010 | BATCH 0657/0938 | LOSS 0.1381 | ACC 0.9588
TRAIN: EPOCH 0010/0010 | BATCH 0658/0938 | LOSS 0.1382 | ACC 0.9589
TRAIN: EPOCH 0010/0010 | BATCH 0659/0938 | LOSS 0.1381 | ACC 0.9589
TRAIN: EPOCH 0010/0010 | BATCH 0660/0938 | LOSS 0.1382 | ACC 0.9589
TRAIN: EPOCH 0010/0010 | BATCH 0661/0938 | LOSS 0.1381 | ACC 0.9589
TRAIN: EPOCH 0010/0010 | BATCH 0662/0938 | LOSS 0.1381 | ACC 0.9588
TRAIN: EPOCH 0010/0010 | BATCH 0663/0938 | LOSS 0.1384 | ACC 0.9588
TRAIN: EPOCH 0010/0010 | BATCH 0664/0938 | LOSS 0.1383 | ACC 0.9589
TRAIN: EPOCH 0010/0010 | BATCH 0665/0938 | LOSS 0.1381 | ACC 0.9589
TRAIN: EPOCH 0010/0010 | BATCH 0666/0938 | LOSS 

TRAIN: EPOCH 0010/0010 | BATCH 0779/0938 | LOSS 0.1394 | ACC 0.9589
TRAIN: EPOCH 0010/0010 | BATCH 0780/0938 | LOSS 0.1393 | ACC 0.9589
TRAIN: EPOCH 0010/0010 | BATCH 0781/0938 | LOSS 0.1392 | ACC 0.9590
TRAIN: EPOCH 0010/0010 | BATCH 0782/0938 | LOSS 0.1392 | ACC 0.9590
TRAIN: EPOCH 0010/0010 | BATCH 0783/0938 | LOSS 0.1393 | ACC 0.9590
TRAIN: EPOCH 0010/0010 | BATCH 0784/0938 | LOSS 0.1393 | ACC 0.9590
TRAIN: EPOCH 0010/0010 | BATCH 0785/0938 | LOSS 0.1392 | ACC 0.9590
TRAIN: EPOCH 0010/0010 | BATCH 0786/0938 | LOSS 0.1391 | ACC 0.9591
TRAIN: EPOCH 0010/0010 | BATCH 0787/0938 | LOSS 0.1390 | ACC 0.9591
TRAIN: EPOCH 0010/0010 | BATCH 0788/0938 | LOSS 0.1390 | ACC 0.9591
TRAIN: EPOCH 0010/0010 | BATCH 0789/0938 | LOSS 0.1388 | ACC 0.9592
TRAIN: EPOCH 0010/0010 | BATCH 0790/0938 | LOSS 0.1396 | ACC 0.9591
TRAIN: EPOCH 0010/0010 | BATCH 0791/0938 | LOSS 0.1399 | ACC 0.9590
TRAIN: EPOCH 0010/0010 | BATCH 0792/0938 | LOSS 0.1398 | ACC 0.9590
TRAIN: EPOCH 0010/0010 | BATCH 0793/0938 | LOSS 

TRAIN: EPOCH 0010/0010 | BATCH 0906/0938 | LOSS 0.1404 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0907/0938 | LOSS 0.1403 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0908/0938 | LOSS 0.1404 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0909/0938 | LOSS 0.1404 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0910/0938 | LOSS 0.1403 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0911/0938 | LOSS 0.1403 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0912/0938 | LOSS 0.1402 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0913/0938 | LOSS 0.1403 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0914/0938 | LOSS 0.1403 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0915/0938 | LOSS 0.1402 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0916/0938 | LOSS 0.1402 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0917/0938 | LOSS 0.1402 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0918/0938 | LOSS 0.1402 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0919/0938 | LOSS 0.1401 | ACC 0.9587
TRAIN: EPOCH 0010/0010 | BATCH 0920/0938 | LOSS 